In [ ]:
#| default_exp losses
%load_ext autoreload
%autoreload 2

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
import os, math, numpy as np
import torch
import torch.nn as nn

class MMD_loss(nn.Module):
    '''
    https://github.com/ZongxianLee/MMD_Loss.Pytorch/blob/master/mmd_loss.py
    '''
    def __init__(self, kernel_mul = 2.0, kernel_num = 5):
        super(MMD_loss, self).__init__()
        self.kernel_num = kernel_num
        self.kernel_mul = kernel_mul
        self.fix_sigma = None
        return
    
    def guassian_kernel(self, source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
        n_samples = int(source.size()[0])+int(target.size()[0])
        total = torch.cat([source, target], dim=0)
        total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
        total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
        L2_distance = ((total0-total1)**2).sum(2) 
        if fix_sigma:
            bandwidth = fix_sigma
        else:
            bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
        bandwidth /= kernel_mul ** (kernel_num // 2)
        bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
        kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]
        return sum(kernel_val)

    def forward(self, source, target):
        batch_size = int(source.size()[0])
        kernels = self.guassian_kernel(source, target, kernel_mul=self.kernel_mul, kernel_num=self.kernel_num, fix_sigma=self.fix_sigma)
        XX = kernels[:batch_size, :batch_size]
        YY = kernels[batch_size:, batch_size:]
        XY = kernels[:batch_size, batch_size:]
        YX = kernels[batch_size:, :batch_size]
        loss = torch.mean(XX + YY - XY -YX)
        return loss

In [ ]:
#| export
import ot
import torch.nn as nn
import torch
import numpy as np
class OT_loss(nn.Module):
    _valid = 'emd sinkhorn sinkhorn_knopp_unbalanced'.split()

    def __init__(self, which='emd', use_cuda=True, reg=0.5, reg_m=0.5):
        if which not in self._valid:
            raise ValueError(f'{which} not known ({self._valid})')
        elif which == 'emd':
            self.fn = lambda m, n, M: ot.emd(m, n, M)
        elif which == 'sinkhorn':
            self.fn = lambda m, n, M : ot.sinkhorn(m, n, M, reg) # maybe use 0.5 for regularization.
        elif which == 'sinkhorn_knopp_unbalanced':
            self.fn = lambda m, n, M : ot.unbalanced.sinkhorn_knopp_unbalanced(m, n, M, reg, reg_m) # keep in mind these regularization parameters and maybe test if they converge.
        else:
            pass
        self.use_cuda=use_cuda

    def __call__(self, source, target, use_cuda=None):
        if use_cuda is None:
            use_cuda = self.use_cuda
        mu = torch.from_numpy(ot.unif(source.size()[0]))
        nu = torch.from_numpy(ot.unif(target.size()[0]))
        M = torch.cdist(source, target)**2
        pi = self.fn(mu, nu, M.detach().cpu())
        if type(pi) is np.ndarray:
            pi = torch.tensor(pi)
        elif type(pi) is torch.Tensor:
            pi = pi.clone().detach()
        pi = pi.cuda() if use_cuda else pi
        M = M.to(pi.device)
        loss = torch.sum(pi * M)
        return loss

In [ ]:
#| export
import torch.nn.functional as F
hard_softmax = F.softmax
# def hard_softmax(input, dim=None):
#     zero_mask = (input.sum(dim=dim, keepdim=True) == 0)
#     input = torch.where(zero_mask, torch.ones_like(input), input)
#     sum_hardsigmoid = torch.sum(input, dim=dim, keepdim=True)
#     softmax_output = input / sum_hardsigmoid
    
#     return softmax_output

# def hard_softmax(input, dim=None, expand_factor=1., shift=0., func=lambda x: x, threshold=None):
#     # func = F.hardsigmoid
#     # Apply hardsigmoid to the input
#     if expand_factor != 1.:
#         input = input * expand_factor
#     if shift != 0.:
#         input = input + shift
#     hardsigmoid_input = func(input)

#     if threshold is not None:
#         hardsigmoid_input = torch.where(hardsigmoid_input < threshold, torch.zeros_like(hardsigmoid_input), hardsigmoid_input)

#     # Identify rows/columns that are all zeros
#     zero_mask = (hardsigmoid_input.sum(dim=dim, keepdim=True) == 0)
    
#     # For rows/columns that are all zeros, set all values to 1
#     hardsigmoid_input = torch.where(zero_mask, torch.ones_like(hardsigmoid_input), hardsigmoid_input)
    
#     # Normalize by the sum along the specified dimension
#     sum_hardsigmoid = torch.sum(hardsigmoid_input, dim=dim, keepdim=True)
#     softmax_output = hardsigmoid_input / sum_hardsigmoid
    
#     return softmax_output

In [ ]:
#| export
import ot
import torch.nn as nn
import torch
import numpy as np
class density_specified_OT_loss(nn.Module):
    """_summary_
    Only allowing balanced Sinkhorn, because it is the only one with stable backprop.

    Args:
        nn (_type_): _description_

    Raises:
        ValueError: _description_

    Returns:
        _type_: _description_
    """
    # _valid = 'emd sinkhorn sinkhorn_knopp_unbalanced'.split()
    _valid = 'sinkhorn'.split()

    def __init__(self, which='sinkhorn', reg=0.1, reg_m=1.0, take_softmax=True, detach_M_ot=False, use_hard_softmax=True, threshold=None):
        if which not in self._valid:
            raise ValueError(f'{which} not known ({self._valid})')
        elif which == 'sinkhorn':
            self.fn = lambda m, n, M : ot.sinkhorn2(m, n, M, reg) # maybe use 0.5 for regularization.
        # elif which == 'sinkhorn_knopp_unbalanced':
        #     self.fn = lambda m, n, M : (ot.unbalanced.sinkhorn_knopp_unbalanced(m, n, M, reg, reg_m) * M).sum() # keep in mind these regularization parameters and maybe test if they converge.
        #     print("WARNING: ot.unbalanced.sinkhorn_knopp_unbalanced chosen -- Can it back propagate density?")
        # elif which == 'emd':
        #     self.fn = lambda m, n, M: ot.emd2(m, n, M)
        #     print("WARNING: ot.emd chosen -- Can it back propagate density?")
        else:
            pass
        if take_softmax:
            if use_hard_softmax:
                self.softmax = lambda x: hard_softmax(x, dim=-1)
            else:
                self.softmax = lambda x: nn.functional.softmax(x, dim=-1)
        else:
            self.softmax = lambda x: x
        self.detach_M_ot = detach_M_ot

    def __call__(self, source, target, source_density, target_density=None):
        # mu = torch.from_numpy(ot.unif(source.size()[0]))
        assert source_density.shape[0] == source.shape[0]
        mu = source_density
        if target_density is None:
            nu = torch.tensor(ot.unif(target.size()[0]), device=source.device, dtype=source.dtype)
        else:
            assert target_density.shape[0] == target.shape[0]
            nu = target_density
        mu = self.softmax(mu)
        nu = self.softmax(nu)
        M = torch.cdist(source, target)#**2 # strangely this squaring prevents backprop through the density. 
        if self.detach_M_ot:
            M = M.detach()
        loss = self.fn(mu, nu, M)
        return loss

In [ ]:
#| export
import torch.nn as nn
import torch
class Density_loss(nn.Module):
    def __init__(self, hinge_value=0.01, use_softmax=True, use_hard_softmax=True):
        self.hinge_value = hinge_value
        if use_softmax:
            if use_hard_softmax:
                self.softmax = lambda x: hard_softmax(x, dim=-1)
            else:
                self.softmax = lambda x: nn.functional.softmax(x, dim=-1)
        pass

    def __call__(self, source, target, groups = None, to_ignore = None, top_k = 5, pre_softmax_weights=None):
        if pre_softmax_weights is not None:
            post_softmax_weights = self.softmax(pre_softmax_weights) * pre_softmax_weights.shape[-1]
        if groups is not None:
            # for global loss
            if pre_softmax_weights is not None:
                c_dist = torch.stack([
                    torch.cdist(source[i], target[i]) * post_softmax_weights[i].unsqueeze(-1)
                    # NOTE: check if this should be 1 indexed
                    for i in range(1,len(groups))
                    if groups[i] != to_ignore
                ])
            else:
                c_dist = torch.stack([
                    torch.cdist(source[i], target[i]) 
                    # NOTE: check if this should be 1 indexed
                    for i in range(1,len(groups))
                    if groups[i] != to_ignore
                ])
        else:
            # for local loss
            if pre_softmax_weights is not None:
                c_dist = torch.stack([
                    torch.cdist(source, target) * post_softmax_weights.unsqueeze(-1)
                ])
            else:
                c_dist = torch.stack([
                    torch.cdist(source, target)                 
                ])
        values, _ = torch.topk(c_dist, top_k, dim=2, largest=False, sorted=False)
        values -= self.hinge_value
        values[values<0] = 0
        loss = torch.mean(values)
        return loss

In [ ]:
#| export
class Local_density_loss(nn.Module):
    def __init__(self):
        pass

    def __call__(self, sources, targets, groups, to_ignore, top_k = 5):
        # print(source, target)
        # c_dist = torch.cdist(source, target) 
        c_dist = torch.stack([
            torch.cdist(sources[i], targets[i]) 
            # NOTE: check if should be from range 1 or not.
            for i in range(1, len(groups))
            if groups[i] != to_ignore
        ])
        vals, inds = torch.topk(c_dist, top_k, dim=2, largest=False, sorted=False)
        values = vals[inds[inds]]
        loss = torch.mean(values)
        return loss

In [ ]:
#| export
class EnergyLossSeq(nn.Module):
    def __init__(self):
        pass

    def __call__(self, func, xtseq, t):
        dxdt = torch.stack([func(t[i], xtseq[i]) for i in range(len(t))])
        return torch.square(dxdt).mean()
    
class EnergyLossGrowthRateSeq(nn.Module):
    def __init__(self, weighted=True, detach_m=False, use_softmax=True, use_hard_softmax=True):
        self.weighted = weighted
        self.detach_m = detach_m
        if use_softmax:
            if use_hard_softmax:
                self.softmax = lambda x: hard_softmax(x, dim=-1)
            else:
                self.softmax = lambda x: nn.functional.softmax(x, dim=-1)
        else:
            self.softmax = lambda x: x

    def __call__(self, func, xtseq, mtseq, t):
        """Returns the energy wrt the x and m curves separately.

        Args:
            func (_type_): _description_
            xtseq (_type_): _description_
            mtseq (_type_): _description_
            t (_type_): _description_

        Returns:
            _type_: _description_
        """
        xmseq = torch.cat([xtseq, mtseq.unsqueeze(-1)], dim=-1)
        dxdt = torch.stack([func(t[i], xmseq[i]) for i in range(len(t))])
        if self.weighted:
            masses = self.softmax(mtseq) * mtseq.shape[-1]
            if self.detach_m:
                masses = masses.detach()
            dxdt = dxdt * masses.unsqueeze(-1) # weighted by mass
        return torch.square(dxdt[...,:-1]).mean(), torch.square(dxdt[...,-1]).mean()
    
class EnergyLoss(nn.Module):
    def __init__(self):
        pass

    def __call__(self, func, x, t): # should use the current t. i.e. t_seq[-1]
        dxdt = func(t, x)
        return torch.square(dxdt).mean()

class EnergyLossGrowthRate(nn.Module):
    def __init__(self, weighted=True, detach_m=False, use_softmax=True, use_hard_softmax=True):
        self.weighted = weighted
        self.detach_m = detach_m
        if use_softmax:
            if use_hard_softmax:
                self.softmax = lambda x: hard_softmax(x, dim=-1)
            else:
                self.softmax = lambda x: nn.functional.softmax(x, dim=-1)
        else:
            self.softmax = lambda x: x

    def __call__(self, func, x, m, t):
        xm = torch.cat([x, m.unsqueeze(-1)], dim=-1)
        dxdt = func(t, xm)
        if self.weighted:
            masses = self.softmax(m) * m.shape[-1]
            if self.detach_m:
                masses = masses.detach()
            dxdt = dxdt * masses.unsqueeze(-1) # weighted by mass
        return torch.square(dxdt[...,:-1]).mean(), torch.square(dxdt[...,-1]).mean()